POSTagging dengan metode klasifikasi konvensional. 

Metode klasifikasi yang digunakan pada contoh ini adalah *Decision Tree*.


sumber: https://nlpforhackers.io/training-pos-tagger/

Import library yang dibutuhkan

In [18]:
import nltk
nltk.download('punkt')
import pandas as pd

from sklearn import tree
from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import Pipeline

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Baca data latih.


In [19]:
colnames = ['Word','Tag']
tsv_read = pd.read_csv('train.01.tsv', sep='\t',names=colnames,header=None)
tsv_read = tsv_read.astype(str)
tsv_read.head()

,Word,Tag
0,Pemerintah,NN
1,bahkan,RB
2,telah,MD
3,mencanangkan,VB
4,dana,NN


Baca dataset menjadi 2 bagian list: kata-kata dalam kalimat dan tag

In [20]:
word_temp = [] 
tag_temp= [] 
training_sentences = []
training_tags = [] 
count_sentences = 0

for index, row in tsv_read.iterrows():   
    word = row['Word'].lower()
    tag = row['Tag']
    key = (word,tag)
    if count_sentences < 50:
        if word != 'nan':
            word_temp.append(word)
            tag_temp.append(tag)
            if word == '.' :
                training_sentences.append(word_temp)
                training_tags.append(tag_temp)
                word_temp = []
                tag_temp = []

    if word == '.' :
        count_sentences += 1      
        if(count_sentences == 50):
            break

In [21]:
print(training_sentences)

[['pemerintah', 'bahkan', 'telah', 'mencanangkan', 'dana', 'untuk', 'memicu', 'sektor', 'usaha', 'kecil', 'dan', 'menengah', 'ukm', 'tumbuh', 'lebih', 'baik', ',', 'karena', 'sektor', 'ini', 'cukup', 'kuat', 'dalam', 'krisis', 'keuangan', 'pada', 'tahun', '1997', 'lalu', ',', 'kata', '-nya', '.'], ['gubernur', 'bank', 'indonesia', 'burhanudin', 'abdullah', 'optimistis', 'mencapai', 'target', 'proyeksi', 'suku', 'bunga', 'indonesia', 'sbi', '2008', 'yang', 'mencapai', '7,5', '-', '8,0', 'persen', '.'], ['badan', 'kredit', 'desa', 'boleh', 'dibilang', 'sebagai', 'tonggak sejarah', 'berdiri', '-nya', 'lembaga', 'keuangan', 'mikro', 'di', 'indonesia', '.'], ['pada', 'tahun', '1991', 'jumlah', 'utang', 'luar negeri', 'pemerintah', 'membengkak', 'dua', 'kali', 'lipat', 'menjadi', 'us', '$', '45,725', 'miliar', '.'], ['kami', 'sampaikan', 'bahwa', 'pemerintah', 'optimis', 'mengenai', 'tercapai', '-nya', 'target', 'pertumbuhan ekonomi', 'sebesar', '6,2', 'persen', 'dalam', 'tahun', '2009', ','

In [22]:
print(training_tags)

[['NN', 'RB', 'MD', 'VB', 'NN', 'SC', 'VB', 'NN', 'NN', 'JJ', 'CC', 'JJ', 'NN', 'VB', 'RB', 'JJ', 'Z', 'SC', 'NN', 'PR', 'RB', 'JJ', 'IN', 'JJ', 'NN', 'IN', 'NN', 'CD', 'CC', 'Z', 'VB', 'PRP', 'Z'], ['NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'JJ', 'VB', 'NN', 'NN', 'NN', 'NN', 'NNP', 'NNP', 'CD', 'SC', 'VB', 'CD', 'Z', 'CD', 'CD', 'Z'], ['NN', 'NN', 'NN', 'MD', 'VB', 'IN', 'NN', 'VB', 'PRP', 'NN', 'NN', 'JJ', 'IN', 'NNP', 'Z'], ['IN', 'NN', 'CD', 'NN', 'NN', 'NN', 'NN', 'VB', 'CD', 'NND', 'VB', 'VB', 'NNP', 'SYM', 'CD', 'CD', 'Z'], ['PRP', 'VB', 'SC', 'NN', 'NN', 'IN', 'VB', 'PRP', 'NN', 'NN', 'JJ', 'CD', 'CD', 'IN', 'NN', 'CD', 'Z', 'VB', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NN', 'NN', 'PR', 'VB', 'IN', 'NN', 'NNP', 'Z', 'SC', 'NN', 'VB', 'NN', 'NN', 'NN', 'NN', 'PR', 'Z', 'VB', 'PRP', 'Z'], ['SC', 'NN', 'NN', 'VB', 'SYM', 'CD', 'CD', 'SC', 'NN', 'NN', 'JJ', 'SYM', 'CD', 'CD', 'Z'], ['NN', 'NNP', 'MD', 'VB', 'SC', 'NN', 'NN', 'NN', 'MD', 'VB', 'Z', 'CD', 'SC', 'VB', 'NN', 'NN', 'SC', '

Fungsi untuk ekstraksi fitur.

Perhatikan bahwa di contoh ini kata tidak diubah menjadi huruf kecil semua (*lower case*), namun juga tidak ada fitur yang terkait dengan penggunaan huruf kapital.

In [23]:
def features(sentence, index):
    """ sentence: [w1, w2, ...], index: the index of the word """
    #print("sentence index = ")
    #print(sentence[index])
    prefix_1 = ''
    prefix_2 = ''
    suffix_1 = ''
    suffix_2 = ''
    if (len(sentence[index])>2):
      prefix_1 = sentence[index][0]
      prefix_2 = sentence[index][:2]
      suffix_1 = sentence[index][-1]
      suffix_2 = sentence[index][-2:]
    return {
        'word': sentence[index],
        'prefix-1': prefix_1,
        'prefix-2': prefix_2,        
        'suffix-1': suffix_1,
        'suffix-2': suffix_2,        
        'prev_word': '' if index == 0 else sentence[index - 1],
        'next_word': '' if index == len(sentence) - 1 else sentence[index + 1],
    }

Fungsi untuk transformasi format dataset

In [24]:
def transform_to_dataset(sentences, tags):
    X, y = [], []
 
    for sentence_idx in range(len(sentences)):
        for index in range(len(sentences[sentence_idx])):
            X.append(features(sentences[sentence_idx], index))
            y.append(tags[sentence_idx][index])
 
    return X, y

Tes pada kalimat uji

In [25]:
colnames = ['Word','Tag']
tsv_read_test = pd.read_csv("test_sentences.tsv",sep="\t", names=colnames, header=None)
tsv_read_test = tsv_read_test.astype(str)
tsv_read_test.head()

,Word,Tag
0,Menteri,NNP
1,pertahanan,NNP
2,AS,NNP
3,dijadwalkan,VB
4,mengunjungi,VB


In [26]:
word_temp = [] 
tag_temp= [] 
testing_sentences = []
testing_tags = [] 

for index, row in tsv_read_test.iterrows():   
    word = row['Word'].lower()
    tag = row['Tag']
    if word_temp != 'nan':
      word_temp.append(word)
      tag_temp.append(tag)
      if word == '.' :
        testing_sentences.append(word_temp)
        testing_tags.append(tag_temp)
        word_temp = []
        tag_temp = []

In [27]:
print(testing_sentences)

[['menteri', 'pertahanan', 'as', 'dijadwalkan', 'mengunjungi', 'india', '.'], ['tata', 'power', 'menyuplai', 'batu bara', 'pada', 'tahun', '2000', '.'], ['pemerintah', 'hati-hati', 'dalam', 'mengelola', 'bumn', '.'], ['perusahaan', 'baru', 'tersebut', 'mencanangkan', 'target', 'perolehan', 'laba bersih', '.'], ['menteri', 'pertahanan', 'mengunjungi', 'pangkalan', 'udara', '.'], ['menurut', 'laporan', 'sekretaris', 'perusahaan', ',', 'laba bersih', 'meningkat', '.'], ['transaksi', 'penjualan', 'barang mewah', 'tahun', '2007', 'turun', '.'], ['menkeu', 'memperkirakan', 'inflasi', 'akan', 'meningkat', 'dibanding', 'tahun', 'lalu', '.'], ['kenaikan', 'tarif', 'didorong', 'oleh', 'target', 'laba bersih', 'yang', 'meningkat', '.'], ['makanan', 'dari', 'luar negeri', 'tidak', 'bisa', 'masuk', 'pasar', 'lokal', '.']]


In [28]:
print(testing_tags)

[['NNP', 'NNP', 'NNP', 'VB', 'VB', 'NNP', 'Z'], ['NNP', 'NNP', 'VB', 'NN', 'IN', 'NN', 'CD', 'Z'], ['NN', 'JJ', 'IN', 'VB', 'NN', 'Z'], ['NN', 'JJ', 'PR', 'VB', 'NN', 'NN', 'NN', 'Z'], ['NNP', 'NNP', 'VB', 'NN', 'NN', 'Z'], ['IN', 'NN', 'NN', 'NN', 'Z', 'NN', 'VB', 'Z'], ['NN', 'NN', 'NN', 'NN', 'CD', 'VB', 'Z'], ['NN', 'VB', 'NN', 'MD', 'VB', 'VB', 'NN', 'CC', 'Z'], ['NN', 'NN', 'VB', 'IN', 'NN', 'NN', 'SC', 'VB', 'Z'], ['NN', 'IN', 'NN', 'NEG', 'MD', 'VB', 'NN', 'JJ', 'Z']]


Training

In [29]:
X, y = transform_to_dataset(training_sentences, training_tags)


from sklearn import tree
from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import Pipeline
 
clf = Pipeline([
    ('vectorizer', DictVectorizer(sparse=False)),
    ('classifier', tree.DecisionTreeClassifier(criterion='entropy'))
])
clf.fit(X, y)   
 
print('Training completed')

Training completed


Testing

Hasil tagging pada kalimat uji

In [30]:
tagging = []
for sentence in testing_sentences:
    tags = clf.predict([features(sentence, index) for index in range(len(sentence))])
    for i in range(len(sentence)):
        print('Word:',sentence[i] ,', Tag:', tags[i])
        tagging.append(tags[i])

Word: menteri , Tag: RB
Word: pertahanan , Tag: NN
Word: as , Tag: NNP
Word: dijadwalkan , Tag: VB
Word: mengunjungi , Tag: VB
Word: india , Tag: NNP
Word: . , Tag: Z
Word: tata , Tag: CD
Word: power , Tag: NNP
Word: menyuplai , Tag: VB
Word: batu bara , Tag: NN
Word: pada , Tag: IN
Word: tahun , Tag: NN
Word: 2000 , Tag: CD
Word: . , Tag: Z
Word: pemerintah , Tag: NN
Word: hati-hati , Tag: JJ
Word: dalam , Tag: IN
Word: mengelola , Tag: NNP
Word: bumn , Tag: NNP
Word: . , Tag: Z
Word: perusahaan , Tag: NN
Word: baru , Tag: JJ
Word: tersebut , Tag: PR
Word: mencanangkan , Tag: VB
Word: target , Tag: NN
Word: perolehan , Tag: NN
Word: laba bersih , Tag: NN
Word: . , Tag: Z
Word: menteri , Tag: RB
Word: pertahanan , Tag: NN
Word: mengunjungi , Tag: VB
Word: pangkalan , Tag: NN
Word: udara , Tag: NN
Word: . , Tag: Z
Word: menurut , Tag: IN
Word: laporan , Tag: NN
Word: sekretaris , Tag: NNP
Word: perusahaan , Tag: NNP
Word: , , Tag: Z
Word: laba bersih , Tag: NN
Word: meningkat , Tag: VB


In [31]:
print(tagging)

['RB', 'NN', 'NNP', 'VB', 'VB', 'NNP', 'Z', 'CD', 'NNP', 'VB', 'NN', 'IN', 'NN', 'CD', 'Z', 'NN', 'JJ', 'IN', 'NNP', 'NNP', 'Z', 'NN', 'JJ', 'PR', 'VB', 'NN', 'NN', 'NN', 'Z', 'RB', 'NN', 'VB', 'NN', 'NN', 'Z', 'IN', 'NN', 'NNP', 'NNP', 'Z', 'NN', 'VB', 'Z', 'NN', 'NN', 'NN', 'NN', 'CD', 'NNP', 'Z', 'RB', 'VB', 'NN', 'MD', 'VB', 'VB', 'NN', 'CC', 'Z', 'NN', 'NN', 'VB', 'IN', 'NN', 'NN', 'SC', 'VB', 'Z', 'NN', 'IN', 'NN', 'NEG', 'NN', 'VB', 'NN', 'JJ', 'Z']


In [32]:
X_test, y_test = transform_to_dataset(testing_sentences, testing_tags)
 
print("Akurasi dengan metode Classification:", clf.score(X_test, y_test))

Akurasi dengan metode Classification: 0.8441558441558441
